In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from k12libs.utils.nb_easy import k12ai_start_html
from urllib.parse import urlencode
import json
import time
import requests

In [3]:
default_code_snip = '''import time, sys
def main():
    """
    中文汉字就是帅, 符号也不赖！＠＃￥％＾＆＊（）＿＋｛｝：＂＞＜？｜
    """
    for i in range(10):
        print(f'{i}'*50, flush=True)
        time.sleep(1)
main()
'''

## API调用(方式1)

In [4]:
host = '116.85.5.40'
port = 8119
user = '16601548608'
uuid = '123456'
token = '123456'
appId = f'k12pyr.{user}.{uuid}'

# API
uri = f'http://{host}:{port}/k12ai/framework/execute'

# 启动 输入:
reqdata = {
    "appId": appId,
    "token": token,
    "op": "runcode.start",      # START
    "user": user,
    "service_name": "k12pyr",
    "service_uuid": uuid,
    "service_params": {
        "code": default_code_snip,
        "devoloper": True
    } 
}

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
# 输出:
# {'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}
resdata

{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}

In [5]:
# 异步数据写redis(key={appId}.runlog) (非正式API)
max_try = 5
cur_cnt = 0

pri = f'http://{host}:{port}/k12ai/private/popmsg?key={appId}.runlog'
while cur_cnt < max_try:
    resdata = json.loads(requests.get(url=pri).text)
    if len(resdata) == 0:
        cur_cnt += 1
        time.sleep(1)
        continue
    cur_cnt = 0
    for item in resdata:
        logdata = json.loads(item)
        if 'data' in logdata and 'expand' in logdata['data']:
            print(logdata['data']['expand']['log'])

################################################################################
                                    RUNNING                                     
################################################################################

00000000000000000000000000000000000000000000000000

11111111111111111111111111111111111111111111111111

22222222222222222222222222222222222222222222222222

33333333333333333333333333333333333333333333333333

44444444444444444444444444444444444444444444444444

55555555555555555555555555555555555555555555555555

66666666666666666666666666666666666666666666666666

77777777777777777777777777777777777777777777777777

88888888888888888888888888888888888888888888888888

99999999999999999999999999999999999999999999999999

################################################################################
                                    FINISHED                                    
##########################################################################

-----------

In [6]:
## 停止 输入:
reqdata = {
    "appId": appId,
    "token": token,
    "op": "runcode.stop",         # STOP
    "user": user,
    "service_name": "k12pyr",
    "service_uuid": uuid,
}
resdata = json.loads(requests.post(url=uri, json=reqdata).text)
resdata

{'code': 100205,
 'message': {'en': 'task is not found or not running',
  'cn': '任务停止错误: 请求的服务不存在或者未启动'}}

## 页面操作(方式2)

In [7]:
params = {
    'default': default_code_snip
}
k12ai_start_html(f'codemirror.html?{urlencode(params)}', width='100%', height='1000px')

In [8]:
runcode = '''from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer, EasyaiDataset
class CustomClassifier(EasyaiClassifier):
    def prepare_dataset(self):
        return self.load_presetting_dataset_('rmnist', '/data/datasets/cv')
trainer = EasyaiTrainer(max_epochs=1, model_summary=None)
trainer.fit(CustomClassifier())
trainer.test()'''